In [1]:
import pandas as pd
import spacy
import os
import glob
import transformers
import torch
import accelerate


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/opt/anaconda3/envs/wolfhaus-python3-9-21/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/opt/anaconda3/envs/wolfhaus-python3-9-21/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/opt/anaconda3/envs/wolfhaus-python3-9-21/lib/python3.9/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/opt/anaconda3/envs/wolfhaus-python3-9-21/lib/python3.9/site-packages/traitlets/config/

In [ ]:
# Define the path to the folder containing the text files
text_folder_path = os.path.join(os.getcwd(), "text")

# Get a list of all text files in the folder
text_files = glob.glob(os.path.join(text_folder_path, "*.txt"))

# Load text files
texts = [open(file, 'r', encoding='utf-8').read() for file in text_files]

In [ ]:
# Define the path to the folder containing the csv files
report_folder_path = os.path.join(os.getcwd(), "report")

# Get a list of all text files in the folder
csv_files = glob.glob(os.path.join(report_folder_path, "*.csv"))
csv_list = []
for file in csv_files:
    csv_df = pd.read_csv(file)
    #drop English
    csv_df.drop('Details (English)', axis='columns', inplace=True)
    #transpose
    csv_df = csv_df.T
    #make first row as column name
    csv_df.columns = csv_df.iloc[0]
    csv_df = csv_df[1:]
    #append to list
    csv_list.append(csv_df)
csv_data = pd.concat(csv_list, ignore_index=True)

In [ ]:
# Load German NLP model
nlp = spacy.load("de_core_news_sm")

# Preprocess texts
def preprocess(text):
    doc = nlp(text.lower())
    tokens = [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]
    return " ".join(tokens)

texts_cleaned = [preprocess(text) for text in texts]
csv_data['cleaned_text'] = texts_cleaned

In [2]:
ROOM_KEYWORDS = [
    "Schlafzimmer", "Küche", "Wohnzimmer", "Badezimmer", "Arbeitszimmer", "Gästezimmer", "Esszimmer", "Kinderzimmer", 
    "Hauptschlafzimmer", "Speisekammer", "Fitnessraum", "Waschraum", "Flur", "Terrasse", "Balkon", "Keller", "Dachboden", 
    "Garage", "Lagerraum", "Spielzimmer", "Heimkino", "Büro", "Sauna", "Gartenhaus", "Hobbyraum", "Wintergarten", 
    "Empfangsraum", "Bibliothek", "Abstellraum", "Medienraum", "Werkstatt", "Technikraum", "Galerie", "Schrankraum", 
    "Kinderbereich", "Gäste-WC", "Dusche", "Umkleideraum", "Weinkeller", "Entspannungsraum", "Lesezimmer", "Pflegezimmer", 
    "Veranda", "Begehbarer Kleiderschrank", "Treppenhaus", "Hauptbadezimmer", "Spielraum", "Offener Wohnbereich", 
    "Doppelgarage", "Multifunktionsraum", "Dachterrasse", "Haupthalle", "Besprechungsraum", "Pförtnerloge", 
    "Außenküche", "Atelier", "Leseecke", "Arbeitsfläche", "Aufenthaltsraum", "Lounge", "Veranstaltungsraum", 
    "Cafeteria", "Bar", "Pavillon", "Behandlungsraum", "Eingangsbereich", "Meditationsraum", "Bastelraum", 
    "Fitnessstudio", "Aufnahmeraum", "Serverraum", "Klassenraum", "Lagerfläche", "Übergangsbereich", "Heizungsraum", 
    "Labor", "Küchenzeile", "Arbeitsinsel", "Speiseraum", "Pausenraum", "Schlafsaal", "Ruheraum", "Tanzstudio", 
    "Empfangsbereich", "Arbeitsstation", "Werkraum", "Außenraum", "Gruppenraum", "Kunststudio", "Rezeption", 
    "Konferenzraum", "Bewegungsraum"
]

SPECIAL_FEATURES_KEYWORDS = [
    "Whirlpool", "Pool", "Sauna", "Terrasse", "Balkon", "Garten", "Feuerstelle", "Kamin", "Fitnessraum", "Heimkino", 
    "Weinkeller", "Außenküche", "Wintergarten", "Veranda", "Bibliothek", "Gewächshaus", "Sichtfenster", "Skylights", 
    "Jacuzzi", "Grillplatz", "Spielplatz", "Trampolin", "Automatisierung", "Smart-Home-System", "Schiebedach", 
    "Lichtkuppel", "Kletterwand", "Fischteich", "Vogelhaus", "Wasserfall", "Kinderspielplatz", "Baumhaus", 
    "Fahrradständer", "Solarpaneele", "Ladestation für Elektroautos", "Whiskeyzimmer", "Billardraum", "Partykeller", 
    "Tanzfläche", "Karaokeraum", "Yoga-Studio", "Beobachtungsdeck", "Klettergerüst", "Boccia-Bahn", "Bowlingbahn", 
    "Tennisplatz", "Golfplatz", "Laufband", "Wellnessbereich", "Behandlungsraum", "Trainingsfläche", 
    "Privater Strand", "Strandkorb", "Indoor-Garten", "Bilderrahmenwand", "Leseecke", "Spielzimmer", "Hobbyraum", 
    "Beamerraum", "Poolhaus", "Offener Kamin", "Naturteich", "Winterlandschaft", "Weinlager", "Gartenschuppen", 
    "Außenfeuerstelle", "Glaswand", "Eingebauter Grill", "Panoramablick", "Wasserfontäne", "Außenbeleuchtung", 
    "Gartenteich", "Kinderrutsche", "Baumhaus", "Außenlounge", "Liegefläche", "Freiluftdusche", "Fass-Sauna", 
    "Natursteinmauer", "Kneippbecken", "Indoor-Pool", "Spielgeräte", "Gartenmöbel", "Pergola", "Hängematte", 
    "Kletterpflanzen", "Kräutergarten", "Ziergarten", "Vogeltränke"
]

DESIGN_STYLE_KEYWORDS = [
    "modern", "rustikal", "minimalistisch", "skandinavisch", "industriell", "klassisch", "traditionell", 
    "zeitgenössisch", "barock", "boho", "luxuriös", "gemütlich", "elegant", "natürlich", "alpin", "ländlich", 
    "antik", "viktorianisch", "kolonialstil", "chalet-stil", "mediterran", "orientalisch", "japanisch", 
    "mid-century modern", "art déco", "futuristisch", "maritim", "loft", "eklektisch", "griechisch", 
    "georgianisch", "kubistisch", "tropisch", "industrial chic", "französisch", "marokkanisch", "scandi-chic", 
    "california style", "landhausstil", "natural chic", "urban modern", "contemporary classic", "retro", 
    "neo-gothisch", "avantgarde", "provencial", "veranda style", "ethnisch", "postmodern", "high-tech", 
    "new-classic", "zen", "country-modern", "kanadisch", "alpenstil", "bohemian", "vintage", "pariser chic", 
    "gartenstil", "blockhausstil", "asiatisch", "coastal chic", "grünraumdesign", "kulturstil", "bali-stil", 
    "italienisch", "australisch", "neoklassisch", "seemannsstil", "alhambra-stil", "organisch", "cottagecore", 
    "hacienda-stil", "norwegisch", "schwedisch", "irisch", "holländisch", "luxury loft"
]

MATERIALS_KEYWORDS = [
    "Holz", "Beton", "Stahl", "Glas", "Ziegel", "Marmor", "Granit", "Kunststoff", "Kupfer", "Aluminium", 
    "Keramik", "Fliesen", "Schiefer", "Naturstein", "Laminat", "PVC", "Lehm", "Bambus", "Eichenholz", 
    "Kiefernholz", "Recyclingmaterialien", "Verbundwerkstoffe", "Carbonfaser", "Fertigbeton", "Kalksandstein", 
    "Faserzement", "Sperrholz", "Edelstahl", "Weichholz", "Hartholz", "Metall", "Kunstharz", "Terrakotta", 
    "Zement", "Dämmstoffe", "Gipskarton", "Porenbeton", "Wellblech", "Furnier", "Gusseisen", "Polycarbonat", 
    "Akustikplatten", "Isolierglas", "Solarzellen", "Quarzit", "Travertin", "Kork", "Basalt", "Schlackestein", 
    "Polymere", "Asphalt", "Dachziegel", "Stroh", "Zementestrich", "Tonziegel", "Eisen", "Textilbeton", 
    "Filz", "Vinyl", "Bitumen", "Styropor", "Glasfaser", "Elastomer", "Papier", "Naturfasern", "Leder", 
    "Biokomposite", "Pressholz", "Epoxidharz", "Hanf", "Jute", "Seide", "Schieferplatten", "Ton", "Blech", 
    "Titan", "Gummimaterialien", "Glaswolle", "Perlite", "Bimsstein", "Polyethylen", "Kunstleder", 
    "Kunstmarmor", "Schweißdraht", "Holzwerkstoffplatten", "Styrodur", "Lichtbeton", "Pigmentierte Farben", 
    "Keramische Fliesen", "Naturmaterialien", "Säulenstein", "Architektenbeton", "Leichtbeton"
]

BUDGET_KEYWORDS = [
    "Euro", "Kosten", "Budget", "Preis", "Finanzierung", "Baukosten", "Projektkosten", "Schätzung", 
    "Investition", "Kostenrahmen", "Kostenvoranschlag", "Bausumme", "Endsumme", "Geldmittel", "Ausgaben", 
    "Aufwand", "Eigenkapital", "Gesamtbudget", "Planungskosten", "Baukredit", "Rohbaukosten", "Materialkosten", 
    "Lohnkosten", "Baufinanzierung", "Kreditrahmen", "Bauzinsen", "Gebühren", "Fördermittel", "Subventionen", 
    "Monatsraten", "Baukreditkosten", "Versicherungskosten", "Rücklagen", "Anzahlung", "Teilsumme", 
    "Gesamtaufwand", "Projektfinanzierung", "Bauabgaben", "Zuschuss", "Abschreibungskosten", "Umsatzsteuer", 
    "Fixkosten", "Vertragssumme", "Abrechnungsbetrag", "Voranschlag", "Eigenleistungskosten", "Planungspuffer", 
    "Projektschätzung", "Baurechnung", "Vorauszahlung", "Steuerkosten", "Gebäudewert", "Schuldendienst", 
    "Sonderkosten", "Kalkulation", "Kreditkosten", "Leasingrate", "Bauzuschuss", "Ersparnisse", "Kapitalbedarf", 
    "Finanzierungsplan", "Abzahlungsplan", "Kostenschätzung", "Projektmittel", "Tilgungsplan", "Hypothek", 
    "Baukostenanalyse", "Kreditbewilligung", "Eigenkapitalquote", "Mittelzuweisung", "Projektfonds", 
    "Rückzahlungsbetrag", "Gewährleistungskosten", "Baukostenübersicht", "Abrechnungsplan", "Schuldzins", 
    "Hauswert", "Bauabsicherung", "Tilgungsbeitrag", "Baukreditangebot", "Bauversicherungsbeitrag", 
    "Baubuchhaltung", "Baumittel", "Zinskosten", "Nebenkosten", "Baudarlehen", "Gesamtausgaben", 
    "Zinsaufwand", "Baupreis", "Materialpreis", "Honorarkosten", "Vertragskosten", "Vertragswert"
]

TIMELINE_KEYWORDS = [
    "Monate", "Jahre", "Wochen", "Zeitrahmen", "Fertigstellung", "Abschluss", "Projektzeit", "Bauzeit", 
    "Termine", "Planungsphase", "Zeitplan", "Meilensteine", "Lieferdatum", "Bauabschluss", "Abgabetermin", 
    "Fertigstellungsdatum", "Frist", "Baukalender", "Projektkalender", "Projektfristen", "Starttermin", 
    "Bauprojektzeit", "Projektstart", "Zieltermin", "Abschlusstermin", "Zeiteinschätzung", "Baufristen", 
    "Projektdauer", "Umsetzungszeit", "Auftragsdauer", "Bearbeitungszeit", "Planungstermine", "Ausführungsphase", 
    "Konstruktionszeit", "Baufertigstellung", "Abnahmefrist", "Lieferfristen", "Verzögerungen", "Endzeit", 
    "Baulaufzeit", "Projektstartdatum", "Projektenddatum", "Zeitabschätzung", "Arbeitszeit", "Projektzyklus", 
    "Bauetappen", "Bauphasen", "Bauverzug", "Projektetappen", "Durchlaufzeit", "Terminvorgabe", "Fertigstellungsfristen", 
    "Fertigstellungsphase", "Zeitschiene", "Bauprojektfristen", "Projektfortschritt", "Vorlaufzeit", 
    "Termineinhaltung", "Abschlusstermine", "Zeithorizont", "Startzeitpunkt", "Endzeitpunkt", "Tagesplan", 
    "Wochenplan", "Zeitmanagement", "Zeitpuffer", "Projektverlauf", "Projektzeitachse", "Bauvorlauf", 
    "Projektverzögerung", "Fertigstellungsdauer", "Projektdeadline", "Fristverlängerung", "Bauende", 
    "Projektende", "Bauabnahme", "Zeitfenster", "Projektmeilensteine", "Projektaufgaben", "Bauplanung", 
    "Lieferzeiträume", "Bautermin", "Zeitvorgaben", "Verzugszeit", "Projektplan", "Zeitrahmenplanung"
]

ADDITIONAL_NOTES_KEYWORDS = [
    "funktional", "nachhaltig", "energieeffizient", "familienfreundlich", "barrierefrei", "umweltfreundlich", 
    "sicher", "modular", "flexibel", "wetterbeständig", "energieautark", "anpassungsfähig", "pflegeleicht", 
    "wartungsarm", "hochwertig", "komfortabel", "innovativ", "raumoptimiert", "modernisiert", "intelligent", 
    "wartungsfreundlich", "ökologisch", "stabil", "langfristig", "ästhetisch", "luxuriös", "minimalistisch", 
    "schalldicht", "lichtdurchflutet", "pflegearm", "zeitgemäß", "belüftet", "hitzeresistent", "schallisoliert", 
    "rauchfrei", "grün", "sicherheitsoptimiert", "praktisch", "stilvoll", "familiengerecht", "einfallsreich", 
    "robust", "widerstandsfähig", "technologisch", "inspirierend", "benutzerfreundlich", "kostenbewusst", 
    "platzsparend", "geräuscharm", "intelligent", "belastbar", "energiesparend", "modular", "hochwertig", 
    "nachhaltigkeit", "autark", "modernisierung", "effizient", "individuell", "umweltgerecht", "pflegeleicht", 
    "langlebig", "naturnah", "gesundheitlich", "vorausschauend", "klimagerecht", "resilient", "raumoptimierung", 
    "innovativer ansatz", "freundlich", "geschmackvoll", "erweiterbar", "harmonisch", "zeitlos", "praktikabel", 
    "schadstoffarm", "dynamisch", "zukunftssicher", "resourcenschonend", "multifunktional", "adaptiv", 
    "wohnfreundlich", "ergonomisch", "modernste technik", "smart", "resistent", "wohnlich"
]

In [3]:
Keywords = ROOM_KEYWORDS + SPECIAL_FEATURES_KEYWORDS + DESIGN_STYLE_KEYWORDS + MATERIALS_KEYWORDS + BUDGET_KEYWORDS + TIMELINE_KEYWORDS +ADDITIONAL_NOTES_KEYWORDS

In [8]:
import csv
# opening the csv file in 'w+' mode
file = open('1000keywords.csv', 'w+', newline ='')
 
# writing the data into the file
with file:    
    write = csv.writer(file)
    write.writerow(Keywords)

In [ ]:
# Updated Feature Extraction Function
def extract_features(text):
    doc = nlp(text)

    # Helper function to match keywords in text
    def match_keywords(keywords, doc_text):
        return ", ".join([word for word in keywords if word.lower() in doc_text.lower()])

    features = {
        "Project Description": text,
        "Rooms": match_keywords(ROOM_KEYWORDS, text),
        "Special Features": match_keywords(SPECIAL_FEATURES_KEYWORDS, text),
        "Design Style": match_keywords(DESIGN_STYLE_KEYWORDS, text),
        "Materials": match_keywords(MATERIALS_KEYWORDS, text),
        "Budget": match_keywords(BUDGET_KEYWORDS, text),
        "Timeline": match_keywords(TIMELINE_KEYWORDS, text),
        "Additional Notes": match_keywords(ADDITIONAL_NOTES_KEYWORDS, text)
    }
    return features

# Extract features for all texts
features_list = [extract_features(text) for text in texts]
features_df = pd.DataFrame(features_list)

# Save the extracted features to a CSV file
#features_df.to_csv('feature/construction_features.csv', index=False)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Use extracted features as input and target from labeled data
X = features_df.drop(columns=['Budget'])  # Remove target from features
y = csv_data['Budget']  # Target column

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Random Forest model
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Evaluate model
print(f"Model Accuracy: {model.score(X_test, y_test)}")

NameError: name 'features_df' is not defined